In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [28]:
raw_df = pd.read_csv('export_2017.csv', encoding = "utf-8")#, nrows = 100000)

In [29]:
raw_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID
0,341999621,80391417,0,06-JAN-17,1311010,62776372,12378259,1.00,61312,19997,...,139428,139428.16 Royal Canin British Shorthair Adult ...,139428.16 Royal Canin British Shorthair Adult ...,Katzenfutter trocken,Royal Canin Rasse (Breed),British Shorthair,British Shorthair,Royal Canin,Royal Canin,3
1,348334117,81849313,0,02-FEB-17,2031175,111951408,2913095,1.00,70828,48693,...,523432,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis","523432.4 Bozita Original - 12 kg + 1,8 kg gratis",Hundefutter trocken,Bozita,Bozita,Bozita,Doggy-Bozita,Dry Food Cat / Dog,2
2,348334114,81849313,0,02-FEB-17,1568093,53626549,2913095,1.00,56851,43068,...,468126,468126.4 Sparpaket Lukullus feine Kauknochen 3...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,Hundesnacks,★ Lukullus,Lukullus feine Kauknochen,Lukullus feine Kauknochen,MATINA,snacks,2
3,354903151,83341895,0,02-MAR-17,1910589,38614848,7097883,0.25,51010,39043,...,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3
4,354903151,83341895,0,02-MAR-17,1910589,38614828,7097883,0.25,51174,39043,...,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3


In [30]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23768830 entries, 0 to 23768829
Data columns (total 22 columns):
SCM_N_ORDER_POS_NR         int64
SCM_N_ORDER_NR             int64
SCM_N_ORDER_NR_NC          int64
TDT_T_KEY_ORDER_DATE       object
ART_N_KEY_ARTICLE          int64
PPR_N_KEY_PHYS_PRODUCT     int64
CUS_N_KEY_CUSTOMER         int64
SCM_D_QUANTITY             float64
PPR_N_PHY_ARTICLE_ID       int64
PPR_N_PHY_PRODUCT_ID       int64
ART_N_KEY_ARTICLE_1        int64
ART_C_ART_NUMBER           float64
ART_C_PROD_NUMBER          int64
ART_V_ART_DESCRIPTION      object
ART_V_ART_DESCRIPTION_1    object
ART_V_PG_LEVEL3_DESC       object
ART_V_PG_LEVEL4_DESC       object
ART_V_PG_LEVEL5_DESC       object
ART_V_PG_LEVEL6_DESC       object
ART_V_MANUFACTURER_DESC    object
PPR_ART_CLASS_DESC         object
PPR_N_LEVEL2_ID            int64
dtypes: float64(2), int64(11), object(9)
memory usage: 3.9+ GB


In [31]:
## should we have any missing customer data
cleaned_retail = raw_df.loc[pd.isnull(raw_df.CUS_N_KEY_CUSTOMER) == False]

In [32]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23768830 entries, 0 to 23768829
Data columns (total 22 columns):
SCM_N_ORDER_POS_NR         int64
SCM_N_ORDER_NR             int64
SCM_N_ORDER_NR_NC          int64
TDT_T_KEY_ORDER_DATE       object
ART_N_KEY_ARTICLE          int64
PPR_N_KEY_PHYS_PRODUCT     int64
CUS_N_KEY_CUSTOMER         int64
SCM_D_QUANTITY             float64
PPR_N_PHY_ARTICLE_ID       int64
PPR_N_PHY_PRODUCT_ID       int64
ART_N_KEY_ARTICLE_1        int64
ART_C_ART_NUMBER           float64
ART_C_PROD_NUMBER          int64
ART_V_ART_DESCRIPTION      object
ART_V_ART_DESCRIPTION_1    object
ART_V_PG_LEVEL3_DESC       object
ART_V_PG_LEVEL4_DESC       object
ART_V_PG_LEVEL5_DESC       object
ART_V_PG_LEVEL6_DESC       object
ART_V_MANUFACTURER_DESC    object
PPR_ART_CLASS_DESC         object
PPR_N_LEVEL2_ID            int64
dtypes: float64(2), int64(11), object(9)
memory usage: 4.1+ GB


Before we make any sort of ratings matrix, it would be nice to have a lookup table that keeps track of each item ID along with a description of that item. Let’s make that now.

In [33]:
item_lookup = cleaned_retail[['ART_C_PROD_NUMBER', 'ART_V_ART_DESCRIPTION']].drop_duplicates() # Only get unique item/description pairs
item_lookup['ART_C_PROD_NUMBER'] = item_lookup.ART_C_PROD_NUMBER.astype(str) # Encode as strings for future lookup ease

In [34]:
item_lookup.head()

,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
0,139428,139428.16 Royal Canin British Shorthair Adult ...
1,523432,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis"
2,468126,468126.4 Sparpaket Lukullus feine Kauknochen 3...
3,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...
5,451643,451643.2 Felix Häppchen in Gelee oder Leckerbi...


This can tell us what each item is, such as that ART_C_PROD_NUMBER 3481 is a  Dog Design Decke Hundeschar . Now that this has been created, we need to:

1. Group purchase quantities together by stock code and item ID
2. Change any sums that equal zero to one (this can happen if items were returned, but we want to indicate that the user actually purchased the item instead of assuming no interaction between the user and the item ever took place)
3. Only include customers with a positive purchase total to eliminate possible errors
4. *Set up our sparse ratings matrix*

This last step is especially important if you don’t want to have unnecessary memory issues! If you think about it, our matrix is going to contain thousands of items and thousands of users with a user/item value required for every possible combination. That is a LARGE matrix, so we can save a lot of memory by keeping the matrix sparse and only saving the locations and values of items that are not zero.

The code below will finish the preprocessing steps necessary for our final ratings sparse matrix:

In [52]:
cleaned_retail['CUS_N_KEY_CUSTOMER '] = cleaned_retail.CUS_N_KEY_CUSTOMER.astype(int) # Convert to int for customer ID
cleaned_retail = cleaned_retail[['ART_C_PROD_NUMBER', 'SCM_D_QUANTITY', 'CUS_N_KEY_CUSTOMER']] # Get rid of unnecessary info
grouped_cleaned = cleaned_retail.groupby(['CUS_N_KEY_CUSTOMER', 'ART_C_PROD_NUMBER']).sum().reset_index() # Group together
grouped_cleaned.SCM_D_QUANTITY.loc[grouped_cleaned.SCM_D_QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('SCM_D_QUANTITY > 3') # Only get customers where purchase totals were positive

## If we look at our final resulting matrix of grouped purchases, we see the following:

grouped_purchased.head()

C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,CUS_N_KEY_CUSTOMER,ART_C_PROD_NUMBER,SCM_D_QUANTITY
0,22,52735,5.0
7,22,397041,4.0
12,22,447409,4.0
50,84,41989,9.0
54,84,160773,9.0


### should I kick out the items with quantity less than 1

In [53]:
customers = list(np.sort(grouped_purchased.CUS_N_KEY_CUSTOMER.unique())) # Get our unique customers
products = list(grouped_purchased.ART_C_PROD_NUMBER.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.SCM_D_QUANTITY) # All of our purchases

rows = grouped_purchased.CUS_N_KEY_CUSTOMER.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = grouped_purchased.ART_C_PROD_NUMBER.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [54]:
purchases_sparse

<472403x6494 sparse matrix of type '<class 'numpy.float64'>'
	with 1375001 stored elements in Compressed Sparse Row format>

In [55]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.9551793569873

# Creating a Training and Validation Set

Typically in Machine Learning applications, we need to test whether the model we just trained is any good on new data it hasn’t yet seen before from the training phase. We do this by creating a test set completely separate from the training set. Usually this is fairly simple: just take a random sample of the training example rows in our feature matrix and separate it away from the training set. That normally looks like this:

With collaborative filtering, that’s not going to work because you need all of the user/item interactions to find the proper matrix factorization. A better method is to hide a certain percentage of the user/item interactions from the model during the training phase chosen at random. Then, check during the test phase how many of the items that were recommended the user actually ended up purchasing in the end. Ideally, you would ultimately test your recommendations with some kind of A/B test or utilizing data from a time series where all data prior to a certain point in time is used for training while data after a certain period of time is used for testing.

For this example, because the time period is only 8 months and because of the purchasing type (products), it is most likely products won’t be purchased again in a short time period anyway. This will be a better test. You can see an example here:

pic

Our test set is an exact copy of our original data. The training set, however, will mask a random percentage of user/item interactions and act as if the user never purchased the item (making it a sparse entry with a zero). We then check in the test set which items were recommended to the user that they ended up actually purchasing. If the users frequently ended up purchasing the items most recommended to them by the system, we can conclude the system seems to be working.

As an additional check, we can compare our system to simply recommending the most popular items to every user (beating popularity is a bit difficult). This will be our baseline.

This method of testing isn’t necessarily the “correct” answer, because it depends on how you want to use the recommender system. However, it is a practical way of testing performance I will use for this example.

Now that we have a plan on how to separate our training and testing sets, let’s create a function that can do this for us. We will also import the random library and set a seed so that you will see the same results as I did.

In [56]:
import random

In [57]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

This will return our training set, a test set that has been binarized to 0/1 for purchased/not purchased, and a list of which users had at least one item masked. We will test the performance of the recommender system on these users only. I am masking 20% of the user/item interactions for this example.

In [58]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

# SLOW Python implementation

# Speeding Up ALS

In [59]:
import implicit

In [60]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead
100%|██████████| 50.0/50 [00:24<00:00,  2.27it/s]


# Evaluating the Recommender System

In [61]:
from sklearn import metrics

In [62]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

Now, utilize this helper function inside of a second function that will calculate the AUC for each user in our training set that had at least one item masked. It should also calculate AUC for the most popular items for our users to compare.

In [63]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

We can now use this function to see how our recommender system is doing. To use this function, we will need to transform our output from the ALS function to csr_matrix format and transpose the item vectors. The original pure Python version output the user and item vectors into the correct format already.

calc_mean_auc(product_train, product_u

In [ ]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system